![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Numerical Methods in Geotechnical Engineering**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics


# Exercise 8 - Coupled Problems, monolithic and staggered schemes

Consider two coupled ODEs:

\begin{align}
    \dot{y} &= k_1 (y_\text{eq}-y)(z_\text{eq}-c_1z)
    \\
    \dot{z} &= k_2 (z_\text{eq}-z)^3 + c_2\dot{y}
\end{align}

First, we import some libraries for numerical basics and for plotting.

In [29]:
import numpy as np
import matplotlib.pyplot as plt

#Some plot settings
plt.style.use('seaborn-deep')
plt.rcParams['lines.linewidth']= 2.0
plt.rcParams['lines.color']= 'black'
plt.rcParams['legend.frameon']=True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['legend.fontsize']=14
plt.rcParams['font.size'] = 14
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.axisbelow'] = True
plt.rcParams['figure.figsize'] = (8, 6)

For simplicity, let's consider a Backward Euler scheme, so that

\begin{align}
    \frac{y_{n+1}- y_n}{t_{n+1} - t_n} &= k_1 (y_\text{eq} - y_{n+1})(z_\text{eq} - c_1 z_{n+1})
    \\
    \frac{z_{n+1}- z_n}{t_{n+1} - t_n} &= k_2 (z_\text{eq} - z_{n+1})^3 + c_2\frac{y_{n+1}- y_n}{t_{n+1} - t_n}
\end{align}

This can be re-cast in a residual form, i.e based on an approximation error:

\begin{align}
    r_y &= \frac{y_{n+1}- y_n}{t_{n+1} - t_n} - k_1 (y_\text{eq} - y_{n+1})(z_\text{eq} - c_1 z_{n+1})
    \\
    r_z &= \frac{z_{n+1}- z_n}{t_{n+1} - t_n} - k_2 (z_\text{eq} - z_{n+1})^3 - c_2 \frac{y_{n+1}- y_n}{t_{n+1} - t_n}
\end{align}

where we would like to find the solutions $y_{n+1}$ and $z_{n+1}$ such that $r_y(y_{n+1},z_{n+1}) = 0$ and $r_z(y_{n+1},z_{n+1}) = 0$ *simultaneously*. To achieve that, we have several options.

## Monolithic scheme

The monolithic scheme iteratively solves both equations simultaneously, i.e. in a monolithic entity. Choosing a Newton-Raphson approach to resolve the non-linearities results in 

\begin{align}
    r_y(y_{n+1},z_{n+1}) &\approx r_y(y_n,z_n) + \left. \frac{\partial r_y}{\partial y} \right|_n \underbrace{(y_{n+1} - y_n)}_{\displaystyle \Delta y_{n+1}} + \left. \frac{\partial r_y}{\partial z} \right|_n \underbrace{(z_{n+1} - z_n)}_{\displaystyle \Delta z_{n+1}} \overset{!}{=} 0
    \\
    r_z(y_{n+1},z_{n+1}) &\approx r_z(y_n,z_n) + \left. \frac{\partial r_z}{\partial y} \right|_n \underbrace{(y_{n+1} - y_n)}_{\displaystyle \Delta y_{n+1}} + \left. \frac{\partial r_z}{\partial z} \right|_n \underbrace{(z_{n+1} - z_n)}_{\displaystyle \Delta z_{n+1}} \overset{!}{=} 0
\end{align}

To make the monolithic structure more clearly visible, let's re-arrange this to $-r_i = J_{ij} \Delta u_j$ form and cast it into a vector-matrix format (making the time step index $n+1$ implicit and introducing the iteration counter $k$):

$$
-
\left(
\begin{array}{c}
    r_y(y_k,z_k)
    \\
    r_z(y_k,z_k)
\end{array}
\right)
=
\left.
\left(
\begin{array}{cc}
    \displaystyle \frac{\partial r_y}{\partial y} & \displaystyle \frac{\partial r_y}{\partial z}
    \\
    \displaystyle \frac{\partial r_z}{\partial y} & \displaystyle \frac{\partial r_z}{\partial z}
\end{array}
\right)
 \right|_k
 \left(
\begin{array}{c}
    \Delta y_{k+1}
    \\
    \Delta z_{k+1}
\end{array}
\right)
= 
\left.
\left(
\begin{array}{cc}
    J_{yy} & J_{yz}
    \\
    J_{zy} & J_{zz}
\end{array}
\right)
 \right|_k
 \left(
\begin{array}{c}
    \Delta y_{k+1}
    \\
    \Delta z_{k+1}
\end{array}
\right)
$$

In [30]:
k_1 = 2.
y_eq = 1.
k_2 = 2
z_eq = 1.

In [31]:
def residual_y(y_i,y_n,z_i,dt,c_1):
    return (y_i - y_n)/dt - k_1 * (y_eq - y_i)*(z_eq - c_1 * z_i)

In [32]:
def residual_z(y_i,y_n,z_i,z_n,dt,c_2):
    return (z_i - z_n)/dt - k_2 * (z_eq - z_i)**3 - c_2 * (y_i - y_n)/dt

In [33]:
def J_yy(z_i,dt, c_1):
    return 1./dt + k_1*(z_eq - c_1 * z_i)

In [34]:
def J_yz(y_i, c_1):
    return k_1 * (y_eq - y_i) * c_1

In [35]:
def J_zz(z_i,dt):
    return 1./dt + 3*k_2*(z_eq - z_i)**2

In [36]:
def J_zy(dt, c_2):
    return -c_2/dt

Now, for every time step we advance the system by applying the generalized midpoint Newton-Raphson update rule. Each new time step value then serves as the initial condition for the following linearization step.

We will *recursively* apply the Newton update until the residual is below a specified tolerance:

$\left| r_{n+1}^{i+1}\right| < \epsilon_\text{abs}$

with a suitably chosen absolute tolerance set here to $10^{-6}$.


In [37]:
def integrate_monolithic(dt,c1,c2):
    #Startwerte
    t_end = 5.
    absolute_tolerance = 1.e-6
    max_iter = 1000
    iteration_counter = np.array([0])
    u = [np.array([0., 0.])]#initial values for y and z
    times = np.array([0.])
    #
    while times[-1]+dt < t_end: #repeat the loop as long as the final time step is below the end point
        times = np.append(times,times[-1]+dt) #here define the next time point as the previous time point plus the time increment dt
        u_old = u[-1] #Starting value for recursive update
        i = 0
        #
        while True:
            #evaluate residual
            res = np.array([residual_y(u_old[0],u[-1][0],u_old[1],dt,c1),
                            residual_z(u_old[0],u[-1][0],u_old[1],u[-1][1],dt,c2)])
            #if residual is below tolerance, above maximum iterations, stop iterations
            if (np.linalg.norm(res) < absolute_tolerance or i > max_iter): 
                break
            #evaluate Jacobian
            Jac_yy = J_yy(u_old[1],dt,c1)
            Jac_yz = J_yz(u_old[0],c1)
            Jac_zy = J_zy(dt,c2)
            Jac_zz = J_zz(u_old[1],dt)
            Jac = np.array([[Jac_yy,Jac_yz],[Jac_zy,Jac_zz]])
            #perform linear step
            u_new = u_old + np.linalg.solve(Jac,-res)
            #update counter
            i += 1
            u_old = u_new #preparation of next recursion

        u.append(u_new) #append the new found solution to the solution vector
        iteration_counter = np.append(iteration_counter,i) #store how much iterations this time step took to converge
        
    return times, u, iteration_counter


In [38]:
from ipywidgets import widgets
from ipywidgets import interact

@interact(dt=widgets.BoundedFloatText(value=0.2,min=1e-3,max=0.8,description='$\Delta t$ / s'))

def plot_monolithic(dt=0.01):
    time_n, u_n, iters11 = integrate_monolithic(dt,1.,1.)
    u11 = np.array(u_n)
    fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(18,12))
    #solution
    ax[0][0].plot(time_n,u11.T[0],marker='d',label='11')
    ax[0][0].legend()
    ax[0][0].set_xlabel('$t$ / s')
    ax[0][0].set_ylabel('$y$')
    #error
    ax[0][1].plot(time_n,u11.T[1],marker='d',label='11')
    ax[0][1].legend()
    ax[0][1].set_xlabel('$t$ / s')
    ax[0][1].set_ylabel('$z$')
    #iterations
    ax[1][0].plot(iters11,marker='d',label='11')
    ax[1][0].set_xlabel('time step')
    ax[1][0].set_ylabel('number of iterations')
    ax[1][0].legend()
    plt.show()

interactive(children=(BoundedFloatText(value=0.2, description='$\\Delta t$ / s', max=0.8, min=0.001), Output()…

# Tasks
* How does the number of iterations change if the system if you a) uncouple the system or b) choose a uni-lateral coupling?
* How does the system evolve for other parameter settings? Try to understand why. Examples: $(c_1,c_2) = (0,2),(1,-5)$

## Staggered scheme

A staggered scheme works by solving one equation for one unknown while keeping the other unknown(s) fixed. With the new solution, one proceeds to the other equation(s) to solve for the next unknown, while keeping the other unknown(s) fixed. In this way, one iterates between the equations until the solution is stationary.

Choosing a Newton-Raphson approach to resolve the non-linearities results in 

\begin{align}
    r_y(y_{n+1},z_n) &\approx r_y(y_n,z_n) + \left. \frac{\partial r_y}{\partial y} \right|_n \underbrace{(y_{n+1} - y_n)}_{\displaystyle \Delta y_{n+1}} \overset{!}{=} 0
    \\
    r_z(y_n,z_{n+1}) &\approx r_z(y_n,z_n) + \left. \frac{\partial r_z}{\partial z} \right|_n \underbrace{(z_{n+1} - z_n)}_{\displaystyle \Delta z_{n+1}} \overset{!}{=} 0
\end{align}

To make the staggered scheme more clearly visible, let's re-arrange this to $-r_i = J_{ij} \Delta u_j$ form, cast it into a vector-matrix format, and introduce the Newton iteration counters $k$ and $l$ as well as the coupling iteration counter $o$:

\begin{align}
            - r_y({}^oy^k_{n+1},{}^o z_{n+1}) &= {}^o J_{yy}|_{n+1}^k \Delta {}^oy_{n+1}^{k+1}
            \\
            - r_z({}^oy_{n+1},{}^o z_{n+1}^k) &= {}^o J_{zz}|_{n+1}^k \Delta {}^oz_{n+1}^{k+1}
\end{align}

The coupling iterations $o$ are the outer loop, while the inner loop consists of the Newton iterations. The overall solution is found when the outer solutions of the individual unknowns $z$ and $y$ no longer change between coupling iterations:

$$
    |{}^oy^k_{n+1} - {}^{o-1}y^k_{n+1}| + |{}^oz^k_{n+1} - {}^{o-1}z^k_{n+1}| < \epsilon_\text{abs}
$$

In [39]:
def integrate_staggered(dt,c1,c2):
    #Startwerte
    t_end = 5.
    absolute_tolerance = 1.e-6
    max_iter = 1000
    iteration_counter = np.array([0])
    y = np.array([0.])#initial values for y
    z = np.array([0.])#initial values for z
    times = np.array([0.])
    #
    while times[-1]+dt < t_end: #repeat the loop as long as the final time step is below the end point
        times = np.append(times,times[-1]+dt) #here define the next time point as the previous time point plus the time increment dt
        y_old = y[-1] #Starting value for recursive update
        z_old = z[-1] #Starting value for recursive update
        y_coup = y_old
        z_coup = z_old
        i = 0
        #
        while True:
            while True:
                #evaluate residual
                res = residual_y(y_old,y[-1],z_old,dt,c1)
                #if residual is below tolerance, above maximum iterations, stop iterations
                if (np.abs(res) < absolute_tolerance or i > max_iter): 
                    break
                #perform linear step
                y_new = y_old - res/J_yy(z_old,dt,c1)
                #update counter
                i += 1
                y_old = y_new #preparation of next recursion

            while True:
                #evaluate residual
                res = residual_z(y_old,y[-1],z_old,z[-1],dt,c2)
                #if residual is below tolerance, above maximum iterations, stop iterations
                if (np.abs(res) < absolute_tolerance or i > max_iter): 
                    break
                #perform linear step
                z_new = z_old - res/J_zz(z_old,dt)
                #update counter
                i += 1
                z_old = z_new #preparation of next recursion

            if ((np.abs(y_coup - y_new) + np.abs(z_coup - z_new)) < absolute_tolerance or i > max_iter):
                break
            y_coup = y_new
            z_coup = z_new

        y = np.append(y,y_new) #append the new found solution to the solution vector
        z = np.append(z,z_new) #append the new found solution to the solution vector
        iteration_counter = np.append(iteration_counter,i) #store how much iterations this time step took to converge
        
    return times, y, z, iteration_counter


In [40]:
from ipywidgets import widgets
from ipywidgets import interact

@interact(dt=widgets.BoundedFloatText(value=0.2,min=1e-3,max=0.8,description='$\Delta t$ / s'))

def plot_staggered(dt=0.01):
    time_n, y, z, iters11 = integrate_staggered(dt,1.,1.)
    fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(18,12))
    #solution
    ax[0][0].plot(time_n,y,marker='d',label='11')
    ax[0][0].legend()
    ax[0][0].set_xlabel('$t$ / s')
    ax[0][0].set_ylabel('$y$')
    #error
    ax[0][1].plot(time_n,z,marker='d',label='11')
    ax[0][1].legend()
    ax[0][1].set_xlabel('$t$ / s')
    ax[0][1].set_ylabel('$z$')
    #iterations
    ax[1][0].plot(iters11,marker='d',label='11')
    ax[1][0].set_xlabel('time step')
    ax[1][0].set_ylabel('number of iterations')
    ax[1][0].legend()
    plt.show()

interactive(children=(BoundedFloatText(value=0.2, description='$\\Delta t$ / s', max=0.8, min=0.001), Output()…

In [41]:
from ipywidgets import widgets
from ipywidgets import interact

@interact(dt=widgets.BoundedFloatText(value=0.2,min=1e-3,max=0.8,description='$\Delta t$ / s'))

def plot_comparison(dt=0.01):
    
    fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(18,12))
    #set values for coupling parameters
    c1 = [0.,1.]
    c2 = [0.,1.,-5.]
    # run all combinations
    for i in range(len(c1)):
        for j in range(len(c2)):
            #run both schemes
            time_n, u_n, iters_m = integrate_monolithic(dt,c1[i],c2[j])
            u_m = np.array(u_n)
            time_n, y_s, z_s, iters_s = integrate_staggered(dt,c1[i],c2[j])
            #plot color
            color = next(ax[0][0]._get_lines.prop_cycler)['color']
            #solution y
            ax[0][0].plot(time_n,u_m.T[0],ls='--',color=color)
            ax[0][0].plot(time_n,y_s,color=color)
            #

            #solution z
            ax[0][1].plot(time_n,u_m.T[1],ls='--',color=color)
            ax[0][1].plot(time_n,z_s,color=color)

            #iterations
            ax[1][0].plot(iters_m,ls='--',color=color)
            ax[1][0].plot(iters_s,color=color)
            
            #dummy
            ax[1][1].plot(0,0,color=color,label="$c_1 = %.1f$, $c_2 = %.1f$" %(c1[i],c2[j]))
    
    ax[0][0].set_xlabel('$t$ / s')
    ax[0][0].set_ylabel('$y$')
    ax[0][1].set_xlabel('$t$ / s')
    ax[0][1].set_ylabel('$z$')
    ax[1][0].set_xlabel('time step')
    ax[1][0].set_ylabel('number of iterations')
    ax[1][1].legend()
    ax[1][1].set_title('monolithic --; staggered __')
    plt.show()

interactive(children=(BoundedFloatText(value=0.2, description='$\\Delta t$ / s', max=0.8, min=0.001), Output()…

## Tasks

* What main difference do you observe between both schemes?
* Run the combination $c_1=-3$, $c_2 = -5$. What do you observe? What happens if you a) increase and b) decrease the time steps?